In [1]:
:!./install.sh

In [2]:
:opt no-lint

# 인터프리터에서 컴파일러로
스택변환함수 개념으로 작성된 인터프리터를 다시 살펴보자.
지난 번 노트북에서는 스택 기본 연산을 정의해서 기본 연산을 이용해서 추상적인 스택을 다루는 방식으로 다른 연산을 정의했다.
스택을 꼭 리스트로만 구현하라는 법은 없기 때문에 추상데이타타입으로 접근하는 것이 차후에 코드 유지보수 측면에서도 바람직할 수도 있다.
하지만 우리가 이걸로 지금 진짜 프로그래머들이 사용할 프로그래밍언어를 구현하자는 것도 아니고
인터프리터와 컴파일러 관련 기본 개념을 실행가능한 코드를 통해 구체적으로 이해하기 위한 것이 주목적이므로, 
여기서는 그냥 코드 길이를 좀 짧게 줄이기 위해서 `addK`같은 연산을 그냥 리스트에 대한 패턴 매칭으로 편하게 작성했다.
지난 번 노트북에서는 연산자 왼쪽부터 먼저 계산되었다는 걸 강조하기 위해 `addLK`라고 했지만 이것도 그냥 짧게 `addK`라고 했다.

In [3]:
data Expr = Val Int
          | Add Expr Expr
          deriving Show
          
type Stack = [Int]
type Kont = Stack -> Stack

haltK :: Kont
haltK s = []

pushK :: Int -> Kont
pushK n s = n : s

addK :: Kont
addK (n2:n1:s) = (n1 + n2) : s

eval :: Expr -> Kont
eval (Val n)     = pushK n
eval (Add e1 e2) = addK . eval e2 . eval e1

In [4]:
eval (Add (Add (Val 2) (Val 3)) (Val 4)) [] -- (2 + 3) + 4

[9]

In [5]:
data Inst = ADD | PUSH Int  deriving Show
type Code = [Inst]

compile :: Expr -> Code
compile (Val n)     = [PUSH n]
compile (Add e1 e2) = compile e1 ++ compile e2 ++ [ADD]

step :: Inst -> Kont
step (PUSH n) = pushK n
step ADD      = addK

run :: Code -> Kont
run []     = haltK
run (c:cs) = run cs . step c

In [6]:
compile (Add (Add (Val 2) (Val 3)) (Val 4))  -- (2 + 3) + 4

[PUSH 2,PUSH 3,ADD,PUSH 4,ADD]

In [7]:
run [PUSH 2,PUSH 3,ADD,PUSH 4,ADD] []

[9]

In [8]:
s0 = []
s1 = step (PUSH 2) s0
s2 = step (PUSH 3) s1
s3 = step ADD      s2
s4 = step (PUSH 4) s3
s5 = step ADD      s4

s0
s1
s2
s3
s4
s5

[]

[2]

[3,2]

[5]

[4,5]

[9]